In [34]:
pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import os
import time


# Define la configuración de la base de datos
database_config = {
    'server': r'DESKTOP-S9274BN\SQLEXPRESS',
    'database': 'nba_henry',
    # 'username': 'your_username',
    # 'password': 'your_password'
}

# Crear la cadena de conexión
connection_string = f"mssql+pyodbc://@{database_config['server']}/{database_config['database']}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Crear el motor de conexión
engine = create_engine(connection_string)



In [2]:
# CODIGO FALLIDO CON CONSULTAS DINAMICAS
class DataHandler(FileSystemEventHandler):
    def __init__(self, engine):
        self.engine = engine
        print("DataHandler initialized.")

    def on_created(self, event):
        print(f"Event detected: {event}")
        if event.is_directory:
            print("The created event is a directory. Ignoring.")
            return None
        elif event.src_path.endswith(".csv"):
            print(f"CSV file detected: {event.src_path}")
            self.process_new_file(event.src_path)

    def process_new_file(self, file_path):
        filename = os.path.basename(file_path)
        print(f"Processing file: {file_path}")
        # Read the new CSV file and drop the 'Unnamed: 0' column if it exists
        new_data = pd.read_csv(file_path)
        if 'Unnamed: 0' in new_data.columns:
            new_data = new_data.drop(columns=['Unnamed: 0'])
        print(f"New data read from {file_path}:\n{new_data.head()}")

        # Load existing data from the database
        existing_data = self.load_existing_data()
        print(f"Existing data loaded from database:\n{existing_data.head()}")

        # Identify new or modified rows
        changes = self.get_changes(existing_data, new_data)
        print(f"Changes identified:\n{changes.head()}")

        # Process and insert the changes into SQL Server
        self.insert_data_into_sql(changes)

    def load_existing_data(self):
        query = "SELECT * FROM team"
        with self.engine.connect() as connection:
            existing_data = pd.read_sql(query, connection)
        return existing_data

    def get_changes(self, existing_data, new_data):
        merged_data = new_data.merge(existing_data, on='id', how='left', suffixes=('', '_existing'))

        # Identify rows that are new or modified
        changed_data = merged_data[merged_data.apply(
            lambda row: any([row[col] != row.get(f"{col}_existing", None) for col in new_data.columns if col != 'id']), axis=1)]
    
        # Drop the '_existing' columns
        changed_data = changed_data[new_data.columns]
        return changed_data

    def insert_data_into_sql(self, data):
        if data.empty:
            print("No new or modified rows to insert.")
            return
        
        with self.engine.connect() as connection:
            # Obtener los nombres de las columnas
            columns = data.columns.tolist()

            for index, row in data.iterrows():
                unique_key = row['id']

                # Verificar si existe el registro
                select_query = text("SELECT COUNT(1) FROM team WHERE id = :id")
                exists = connection.execute(select_query, {'id': unique_key}).scalar()

                if exists:
                    # Construir la consulta UPDATE dinámicamente
                    update_columns = ", ".join([f"{col} = :{col}" for col in columns if col != 'id'])
                    update_values = {col: row[col] for col in columns if col != 'id'}
                    update_values['id'] = unique_key
                    update_query = text(f"UPDATE team SET {update_columns} WHERE id = :id")
                    connection.execute(update_query, update_values)
                    print('Update')
                    print(f"Row with unique key {unique_key} updated.")
                else:
                    # Construir la consulta INSERT dinámicamente
                    insert_columns = ", ".join(columns)
                    insert_placeholders = ", ".join([f":{col}" for col in columns])
                    insert_values = {col: row[col] for col in columns}
                    insert_query = text(f"INSERT INTO team ({insert_columns}) VALUES ({insert_placeholders})")
                    print(insert_query)
                    connection.execute(insert_query, insert_values)
                    print('Insert')
                    print(connection_string)
                    print(f"Row with unique key {unique_key} inserted.")

        print("Data inserted/updated successfully.")

In [37]:
#CODIGO SIMULACRO EXITOSO
class DataHandler(FileSystemEventHandler):
    def __init__(self, engine):
        self.engine = engine
        print("DataHandler initialized.")

    def on_created(self, event):
        print(f"Event detected: {event}")
        if event.is_directory:
            print("The created event is a directory. Ignoring.")
            return None
        elif event.src_path.endswith(".csv"):
            print(f"CSV file detected: {event.src_path}")
            self.process_new_file(event.src_path)

    def process_new_file(self, file_path):
        filename = os.path.basename(file_path)
        print(f"Processing file: {file_path}")
        # Read the new CSV file and drop the 'Unnamed: 0' column if it exists
        new_data = pd.read_csv(file_path)
        if 'Unnamed: 0' in new_data.columns:
            new_data = new_data.drop(columns=['Unnamed: 0'])
        print(f"New data read from {file_path}:\n{new_data.head()}")

        # Load existing data from the database
        existing_data = self.load_existing_data()
        print(f"Existing data loaded from database:\n{existing_data.head()}")

        # Identify new or modified rows
        changes = self.get_changes(existing_data, new_data)
        print(f"Changes identified:\n{changes.head()}")

        # Process and insert the changes into SQL Server
        self.insert_data_into_sql(changes)

    def load_existing_data(self):
        query = "SELECT * FROM team"
        with self.engine.connect() as connection:
            existing_data = pd.read_sql(query, connection)
        return existing_data

    def get_changes(self, existing_data, new_data):
        merged_data = new_data.merge(existing_data, on='id', how='left', suffixes=('', '_existing'))

        # Identify rows that are new or modified
        changed_data = merged_data[merged_data.apply(
            lambda row: any([row[col] != row.get(f"{col}_existing", None) for col in new_data.columns if col != 'id']), axis=1)]
    
        # Drop the '_existing' columns
        changed_data = changed_data[new_data.columns]
        return changed_data

    def insert_data_into_sql(self, data):
        if data.empty:
            print("No new or modified rows to insert.")
            return
        
        with self.engine.connect() as connection:
            # Obtener los nombres de las columnas
            data.to_sql('team', connection, if_exists='append', index=False)

In [4]:
class DataHandler(FileSystemEventHandler):
    def __init__(self, engine):
        self.engine = engine
        print("DataHandler initialized.")

    def on_created(self, event):
        print(f"Event detected: {event}")
        if event.is_directory:
            print("The created event is a directory. Ignoring.")
            return None
        elif event.src_path.endswith(".csv"):
            print(f"CSV file detected: {event.src_path}")
            self.process_new_file(event.src_path)

    def process_new_file(self, file_path):
        filename = os.path.basename(file_path)
        print(f"Processing file: {file_path}")
        # Read the new CSV file and drop the 'Unnamed: 0' column if it exists
        new_data = pd.read_csv(file_path)
        if 'Unnamed: 0' in new_data.columns:
            new_data = new_data.drop(columns=['Unnamed: 0'])
        print(f"New data read from {file_path}:\n{new_data.head()}")

        # Load existing data from the database
        existing_data = self.load_existing_data(filename)
        print(f"Existing data loaded from database:\n{existing_data.head()}")

        # Identify new or modified rows
        changes = self.get_changes(existing_data, new_data,filename)
        print(f"Changes identified:\n{changes.head()}")

        # Process and insert the changes into SQL Server
        self.insert_data_into_sql(changes,filename)

    def load_existing_data(self,filename):
        print(filename + " load existing data")
      
        filename_without_extension = filename.replace(".csv", "")
        
        query = f"SELECT * FROM {filename_without_extension}"
        print(query)
        
        with self.engine.connect() as connection:
            existing_data = pd.read_sql(query, connection)
        return existing_data

    def get_changes(self, existing_data, new_data,filename):
        print(filename + " get changes")
        merged_data = new_data.merge(existing_data, on='id', how='left', suffixes=('', '_existing'))

        # Identify rows that are new or modified
        changed_data = merged_data[merged_data.apply(
            lambda row: any([row[col] != row.get(f"{col}_existing", None) for col in new_data.columns if col != 'id']), axis=1)]
    
        # Drop the '_existing' columns
        changed_data = changed_data[new_data.columns]
        return changed_data

    def insert_data_into_sql(self, data,filename):
        filename_without_extension = filename.replace(".csv", "")
        
        if data.empty:
            print("No new or modified rows to insert.")
            return
        
        with self.engine.connect() as connection:
            # Obtener los nombres de las columnas
            data.to_sql(filename_without_extension, connection, if_exists='append', index=False)

In [5]:
# Define la función principal para iniciar el observador
if __name__ == "__main__":
    path_to_watch = 'data/watchdog_carpeta'  # Cambia esto a tu directorio real
    print(f"Starting to watch directory: {path_to_watch}")
    event_handler = DataHandler(engine)
    observer = Observer()
    observer.schedule(event_handler, path=path_to_watch, recursive=False)
    observer.start()
    try:
        while True:
            time.sleep(1)  # Mantén el script en ejecución
    except KeyboardInterrupt:
        observer.stop()
    observer.join()


Starting to watch directory: data/watchdog_carpeta
DataHandler initialized.
Event detected: FileCreatedEvent(src_path='data/watchdog_carpeta\\game_info.csv', dest_path='', event_type='created', is_directory=False, is_synthetic=False)
CSV file detected: data/watchdog_carpeta\game_info.csv
Processing file: data/watchdog_carpeta\game_info.csv
New data read from data/watchdog_carpeta\game_info.csv:
    id   game_date  attendance
0  101  2023-06-21     15000.0
1  102  2023-06-22     20000.0
2  103  2023-06-23     18000.0
3  104  2023-06-24     22000.0
game_info.csv load existing data
SELECT * FROM game_info
Existing data loaded from database:
         id   game_date  attendance
0  11300001  2013-10-05       12191
1  11300002  2013-10-05       15273
2  11300003  2013-10-05       15049
3  11300005  2013-10-06       13538
4  11300006  2013-10-06       16722
game_info.csv get changes
Changes identified:
    id   game_date  attendance
0  101  2023-06-21     15000.0
1  102  2023-06-22     20000.0